In [93]:
import pandas as pd
from datetime import datetime, timedelta
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from dateutil import parser
import numpy as np
import joblib
loaded_model = joblib.load('./regrmodel.joblib')

token = "JxCGieLiMyN7lNeNnOiPQstwDdw3-diYqbDQvfbVz9lohjSVvGqFbvcqp2xN_s5TCgaYi4koju3TZhmZE-WrxQ=="
org = "lab"
url = "http://10.118.126.244:8086/"
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org) #set client
query_api = client.query_api()
query = """from(bucket: "ESP32")
|> range(start: -14d, stop:now())
|> filter(fn: (r) => r["_measurement"] == "Energy_status")
|> filter(fn: (r) => r["_field"] == "W")
|> timeShift(duration: 8h)
|> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
|> drop(columns: ["_start","_stop","_measurement","_field"])
|> top(n: 1440, columns: ["_time"])
|> sort(columns: ["_time"])
|> yield(name: "mean")"""


In [94]:
data_frame = query_api.query_data_frame(query, org="lab")
test_data=data_frame.drop(["result","table"], axis=1)
print(test_data)

c:\Users\Nic\anaconda3\envs\python310\lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "ESP32")
|> range(start: -14d, stop:now())
|> filter(fn: (r) => r["_measurement"] == "Energy_status")
|> filter(fn: (r) => r["_field"] == "W")
|> timeShift(duration: 8h)
|> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
|> drop(columns: ["_start","_stop","_measurement","_field"])
|> top(n: 1440, columns: ["_time"])
|> sort(columns: ["_time"])
|> yield(name: "mean") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in

                         _time  _value
0    2022-11-30 14:42:00+00:00   35.90
1    2022-11-30 14:43:00+00:00   36.60
2    2022-11-30 14:44:00+00:00   36.40
3    2022-11-30 14:45:00+00:00   37.12
4    2022-11-30 14:46:00+00:00   35.76
...                        ...     ...
1435 2022-12-04 00:35:00+00:00    4.85
1436 2022-12-04 00:37:00+00:00   21.92
1437 2022-12-04 00:38:00+00:00   30.84
1438 2022-12-04 00:39:00+00:00   30.24
1439 2022-12-05 17:14:00+00:00   26.39

[1440 rows x 2 columns]


In [95]:
data=pd.DataFrame(test_data)
data.dtypes

_time     datetime64[ns, tzutc()]
_value                    float64
dtype: object

In [96]:
data[['_time']]=data[['_time']].astype('string')
month = []
day = []
hour = []
minute = []
for i in range(len(data)):
    month.append(parser.parse(data['_time'][i]).month)
    day.append(parser.parse(data['_time'][i]).day)
    hour.append(parser.parse(data['_time'][i]).hour)
    minute.append(parser.parse(data['_time'][i]).minute)
data = data.drop('_time',axis = 1)
data.insert(loc=0, column='minute', value=minute)
data.insert(loc=0, column='hour', value=hour)
data.insert(loc=0, column='day', value=day)
data.insert(loc=0, column='month', value=month)

In [97]:
data

,month,day,hour,minute,_value
0,11,30,14,42,35.90
1,11,30,14,43,36.60
2,11,30,14,44,36.40
3,11,30,14,45,37.12
4,11,30,14,46,35.76
...,...,...,...,...,...
1435,12,4,0,35,4.85
1436,12,4,0,37,21.92
1437,12,4,0,38,30.84
1438,12,4,0,39,30.24


In [101]:
data

,month,day,hour,minute,_value
0,11,30,14,42,35.90
1,11,30,14,43,36.60
2,11,30,14,44,36.40
3,11,30,14,45,37.12
4,11,30,14,46,35.76
...,...,...,...,...,...
1435,12,4,0,35,4.85
1436,12,4,0,37,21.92
1437,12,4,0,38,30.84
1438,12,4,0,39,30.24


In [115]:
new_data = []
temp = 0
count= 0
for i in range(len(data)):
    count+=1
    temp += data['_value'][i]
    if(count==60):
        hour_data = []
        hour_data.append(data['month'][len(new_data)*60])
        hour_data.append(data['day'][len(new_data)*60])
        hour_data.append(temp)
        new_data.append(hour_data)
        count = 0
        temp=0

In [121]:
print(new_data)

[[11, 30, 2193.5399999999995], [11, 30, 2197.615], [11, 30, 2183.23], [11, 30, 2164.9553333333333], [11, 30, 2160.5149999999994], [11, 30, 2160.2150000000006], [11, 30, 2151.8950000000004], [11, 30, 2141.4700000000003], [11, 30, 2136.935], [11, 30, 2114.4599999999996], [12, 1, 2111.0099999999998], [12, 1, 2113.475], [12, 1, 2112.91], [12, 1, 2115.825], [12, 1, 2118.705000000001], [12, 1, 2123.675], [12, 1, 2112.97], [12, 1, 2148.8099999999995], [12, 1, 2165.9499999999994], [12, 1, 2190.135], [12, 1, 2171.4249999999997], [12, 1, 2152.2399999999993], [12, 1, 2166.48], [12, 1, 1588.0565179487178]]


In [126]:
x = np.empty([len(new_data), 26])
y = np.empty([len(new_data), 1])
for i in range(x.shape[0]):
    x[i][0] = new_data[i][0]
    x[i][1] = new_data[i][1]
    for j in range(24):
        x[i][2+j] = new_data[i][2]
    y[i] = new_data[i][2]

In [127]:
x

array([[1.10000000e+01, 3.00000000e+01, 2.19354000e+03, 2.19354000e+03,
        2.19354000e+03, 2.19354000e+03, 2.19354000e+03, 2.19354000e+03,
        2.19354000e+03, 2.19354000e+03, 2.19354000e+03, 2.19354000e+03,
        2.19354000e+03, 2.19354000e+03, 2.19354000e+03, 2.19354000e+03,
        2.19354000e+03, 2.19354000e+03, 2.19354000e+03, 2.19354000e+03,
        2.19354000e+03, 2.19354000e+03, 2.19354000e+03, 2.19354000e+03,
        2.19354000e+03, 2.19354000e+03],
       [1.10000000e+01, 3.00000000e+01, 2.19761500e+03, 2.19761500e+03,
        2.19761500e+03, 2.19761500e+03, 2.19761500e+03, 2.19761500e+03,
        2.19761500e+03, 2.19761500e+03, 2.19761500e+03, 2.19761500e+03,
        2.19761500e+03, 2.19761500e+03, 2.19761500e+03, 2.19761500e+03,
        2.19761500e+03, 2.19761500e+03, 2.19761500e+03, 2.19761500e+03,
        2.19761500e+03, 2.19761500e+03, 2.19761500e+03, 2.19761500e+03,
        2.19761500e+03, 2.19761500e+03],
       [1.10000000e+01, 3.00000000e+01, 2.18323000e+03

In [131]:
y1=loaded_model.predict(x)

In [132]:
y1[0]

2146.1178166667005

In [51]:
'''for i in range(len(pd1)):
    time_format=pd1.loc[i,'_time'].strftime('%Y-%m-%dT%H:%M:%SZ')
    pd1.loc[i,'_time']=time_format'''

2022-11-30 14:42:00+00:00
2022-11-30 14:43:00+00:00
2022-11-30 14:44:00+00:00
2022-11-30 14:45:00+00:00
2022-11-30 14:46:00+00:00
2022-11-30 14:47:00+00:00
2022-11-30 14:48:00+00:00
2022-11-30 14:49:00+00:00
2022-11-30 14:50:00+00:00
2022-11-30 14:51:00+00:00
2022-11-30 14:52:00+00:00
2022-11-30 14:53:00+00:00
2022-11-30 14:54:00+00:00
2022-11-30 14:55:00+00:00
2022-11-30 14:56:00+00:00
2022-11-30 14:57:00+00:00
2022-11-30 14:58:00+00:00
2022-11-30 14:59:00+00:00
2022-11-30 15:00:00+00:00
2022-11-30 15:01:00+00:00
2022-11-30 15:02:00+00:00
2022-11-30 15:03:00+00:00
2022-11-30 15:04:00+00:00
2022-11-30 15:05:00+00:00
2022-11-30 15:06:00+00:00
2022-11-30 15:07:00+00:00
2022-11-30 15:08:00+00:00
2022-11-30 15:09:00+00:00
2022-11-30 15:10:00+00:00
2022-11-30 15:11:00+00:00
2022-11-30 15:12:00+00:00
2022-11-30 15:13:00+00:00
2022-11-30 15:14:00+00:00
2022-11-30 15:15:00+00:00
2022-11-30 15:16:00+00:00
2022-11-30 15:17:00+00:00
2022-11-30 15:18:00+00:00
2022-11-30 15:19:00+00:00
2022-11-30 1